In [1]:
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud, plot_3D

from PIL import Image
from tqdm.auto import tqdm
import plotly.graph_objects as go

from point_e.util.point_cloud import PointCloud


In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
%cd ..
!pip install -e .

c:\Users\X1G6\OneDrive - Hanoi University of Science and Technology\FPT_AIC\code
^C


Obtaining file:///C:/Users/X1G6/OneDrive%20-%20Hanoi%20University%20of%20Science%20and%20Technology/FPT_AIC/code


ERROR: file:///C:/Users/X1G6/OneDrive%20-%20Hanoi%20University%20of%20Science%20and%20Technology/FPT_AIC/code does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

creating base model...
creating upsample model...
downloading base checkpoint...
downloading upsampler checkpoint...


<All keys matched successfully>

In [ ]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 0.0],
    model_kwargs_key_filter=('texts', ''), # Do not condition the upsampler at all
)

In [5]:
sampler = PointCloudSampler(
  device=device,
  models=[base_model],
  diffusions=[base_diffusion],
  num_points=[1024],
  aux_channels=['R', 'G', 'B'],
  guidance_scale=[3.0],
  model_kwargs_key_filter=['texts'],
  use_karras = [True],
  karras_steps = [64],
  sigma_min = [1e-3],
  sigma_max = [120],
  s_churn = [3],
)

In [11]:
print(samples)

tensor([[[-1.4755e-02,  8.3027e-03,  2.0180e-03,  ...,  8.1520e-03,
          -6.8459e-02, -2.8924e-02],
         [ 2.6321e-02,  3.2393e-02,  8.3271e-02,  ...,  4.9217e-02,
          -4.6284e-02,  8.7147e-03],
         [-2.3000e-01, -2.5585e-01, -1.5788e-01,  ..., -3.8034e-01,
          -1.5378e-01, -1.7454e-01],
         [ 1.6914e+02,  1.5722e+02,  1.4277e+02,  ...,  1.3945e+02,
           1.6102e+02,  1.6777e+02],
         [ 3.9010e+01,  9.5830e+00,  3.6495e+01,  ...,  4.5379e+00,
           2.3765e+01,  2.8359e+01],
         [-2.0927e+00,  1.1388e+01, -5.7164e+00,  ..., -2.4936e+01,
          -3.7699e+00, -1.7856e+01]]])


In [8]:
# Set a prompt to condition on.
prompt = 'a red motorcycle'

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
    samples = x

0it [00:00, ?it/s]

TypeError: karras_sample_progressive() missing 1 required positional argument: 'diffusion'

In [4]:
# Set a prompt to condition on.
prompt = 'a dog' # @param {type:"string"}

# Produce a sample from the model.
samples = None
iterSample = []
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(texts=[prompt]))):
  samples = x
  iterSample.append(x)


NameError: name 'sampler' is not defined

In [16]:
iterSample[4]

tensor([[[-4.4523e-02, -1.6998e-02,  7.4959e-04,  ...,  5.8441e-03,
          -1.2627e-02, -9.4004e-02],
         [-5.1922e-02, -6.1807e-02, -6.6776e-02,  ..., -1.4420e-03,
          -1.3981e-02, -2.3487e-02],
         [-5.7757e-02, -9.6827e-03, -8.2583e-03,  ..., -1.2584e-02,
          -3.1506e-02, -6.1024e-02],
         [ 1.6500e+02,  1.4112e+02,  1.3735e+02,  ...,  1.3462e+02,
           1.7511e+02,  1.8974e+02],
         [ 9.6765e+01,  1.0524e+02,  9.4439e+01,  ...,  1.0851e+02,
           1.1414e+02,  1.5013e+02],
         [ 1.0068e+02,  1.0992e+02,  9.2274e+01,  ...,  1.1387e+02,
           1.2551e+02,  1.4588e+02]]])

In [21]:
pc = PointCloud.load('point_e\examples\example_data\pc_corgi.npz')


# Set a prompt to condition on.
prompt = 'a corgi' # @param {type:"string"}

# Produce a sample from the model.
samples = None
iterSample = []
for x in tqdm(sampler.invert(pc, model_kwargs=dict(texts=[prompt]))):
  samples = x
  iterSample.append(x)


0it [00:00, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\X1G6\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\X1G6\AppData\Local\Temp\ipykernel_17224\2451079390.py", line 10, in <module>
    for x in tqdm(sampler.invert(pc, model_kwargs=dict(texts=[prompt]))):
  File "c:\Users\X1G6\anaconda3\Lib\site-packages\tqdm\notebook.py", line 254, in __iter__
    for obj in it:
  File "c:\Users\X1G6\anaconda3\Lib\site-packages\tqdm\std.py", line 1178, in __iter__
    for obj in iterable:
  File "c:\Users\X1G6\OneDrive - Hanoi University of Science and Technology\FPT_AIC\code\point-e\point_e\diffusion\sampler.py", line 316, in invert
    for x in samples_it:
  File "c:\Users\X1G6\OneDrive - Hanoi University of Science and Technology\FPT_AIC\code\point-e\point_e\diffusion\k_diffusion.py", line 181, in karras_sample_progressive
    for obj in sample_fn(
  File "c:\Users\X1G6\anaconda3\Lib\site-pa

In [17]:
pc = sampler.output_to_point_clouds(iterSample[4])[0]
# plot_3D(pc)
tensor = pc.to_tensor()
tensor

tensor([[[-4.4523e-02, -1.6998e-02,  7.4959e-04,  ...,  5.8441e-03,
          -1.2627e-02, -9.4004e-02],
         [-5.1922e-02, -6.1807e-02, -6.6776e-02,  ..., -1.4420e-03,
          -1.3981e-02, -2.3487e-02],
         [-5.7757e-02, -9.6827e-03, -8.2583e-03,  ..., -1.2584e-02,
          -3.1506e-02, -6.1024e-02],
         [ 1.6500e+02,  1.4100e+02,  1.3700e+02,  ...,  1.3500e+02,
           1.7500e+02,  1.9000e+02],
         [ 9.7000e+01,  1.0500e+02,  9.4000e+01,  ...,  1.0900e+02,
           1.1400e+02,  1.5000e+02],
         [ 1.0100e+02,  1.1000e+02,  9.2000e+01,  ...,  1.1400e+02,
           1.2600e+02,  1.4600e+02]]])

In [8]:
sampler

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))